# Algorithme de classification des billets de banque

Cet algorithme de classification a été élaboré à partir d'un échantillon de 170 observations, pour rappel fichier .csv transmis par la PJ. Les données ont été retraitées, suivi d'une première étape de réduction de dimension non-supervisée par ACP (*la plus connue des méthodes factorielles*), puis par du clustering KMeans pour enfin pouvoir entraîner et optimiser un modèle de classification binaire, la regression logistique.

In [7]:
#Import des librairies Python
import pandas as pd
from sklearn import preprocessing
import pickle

### Fichier csv à tester

In [8]:
#Jeu de données à stocker dans le dossier "p6_03_dataset/…"
mydata = 'p6_data/test_notes-1.csv'

In [9]:
#Aperçu rapide du jeu de données
pd.read_csv(mydata)

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [10]:
#Fonction "detecteur_billet()" à utiliser dans l'étape suivante
def detecteur_billet(mydata):
    """ Programme de détection de faux billets à partir d'un fichier .csv
        ici 'mydata' 
    """
    
    # Chargement et préparation des données 
    df = pd.read_csv(mydata)
    X = df.copy()
    X = X.iloc[:, :-1]
    
    # Centrage/réduction des données (éviter les écarts d'échelle, les données prendront toutes la même importance)
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
    
    # Récupération du modèle de Régression Logistique 'lr'
    with open('p6_data/model.pkl', 'rb') as f:
        my_unpickler = pickle.Unpickler(f)
        lr = my_unpickler.load()
    
    # Utilisation du modèle de prédiction 'lr'
    probability = lr.predict_proba(X_scaled)[:, 1]
    
    # Probabilités des billets établies 
    proba = pd.Series(probability.round(3), name='%value')
    
    # Intégration des probabilités dans le jeu de données
    df_final = pd.concat([df, proba], axis=1)
    
    # Résultats de la classification prédictive, selon une probabilité  supérieure ou égale à 0.5 :
    resultat = []
    for i in df_final['%value'] >= .5:
        if i is True :
            resultat.append('Vrai Billet')
        else :
            resultat.append('Faux Billet')

    df_final['resultat'] = resultat

    return df_final
    
    

### Lancement de l'algorithme
Pour chaque billet, l'algorithme de classification donnera la probabilité que le billet soit vrai. Si cette probabilité est supérieure ou égale à 0.5, le billet sera considéré comme vrai. Dans le cas contraire, il sera considéré comme faux.

In [11]:
#Dernière étape avant l'affichage des prédictions 
detecteur_billet(mydata)

,diagonal,height_left,height_right,margin_low,margin_up,length,id,%value,resultat
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,0.998,Vrai Billet
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,0.057,Faux Billet
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,1.000,Vrai Billet
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,0.000,Faux Billet
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,0.022,Faux Billet


**Merci d'avoir utilisé cet algorithme de classification…**